<a href="https://colab.research.google.com/github/rexbrandy/cifar10_classification/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DataLoaders

In [8]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def get_dataloaders(batch_size=5, show_images=True):

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    train_dataset = datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )

    test_dataset = datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    if show_images:
        visualize_data(train_dataset)

    return train_dataloader, test_dataloader

In [6]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_data(training_data):
    figure = plt.figure(figsize=(8, 8))
    cols, rows = 5, 2
    for i in range(1, cols * rows + 1):

        sample_idx = torch.randint(len(training_data), size=(1,)).item()

        img, label = training_data[sample_idx]

        figure.add_subplot(rows, cols, i)
        plt.title(CLASSES[label])
        plt.axis("off")

        img = img / 2 + 0.5
        np_img = np.transpose(img, (1, 2, 0))
        plt.imshow(np_img)
    plt.show()